In [56]:
import itertools
import numpy as np
import math
from dismal import generator_matrices_ext
from scipy.stats import poisson
from dismal.generator_matrices import GeneratorMatrix
from scipy import linalg

Define generator matrices

In [11]:
mig_mat1 = np.array(
    [
        [0, 0.1],
        [0.1, 0]
    ]
)


popsizes1 = [1, 1]

np.set_printoptions(precision=3)
q1 = generator_matrices_ext.q(mig_mat1, popsizes1)

In [12]:
mig_mat2 = np.array(
    [
        [0, 0],
        [0, 0]
    ]
)


popsizes2 = [1, 1]

np.set_printoptions(precision=3)
q2 = generator_matrices_ext.q(mig_mat2, popsizes2)

In [13]:
q2

array([[-1.,  0.,  0.,  1.],
       [ 0., -1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [16]:
q3 = np.array(
    [[-1, 0, 0, 1],
    [0, -1, 0, 1],
    [0, 0, -1, 1],
    [0, 0, 0, 0]]
)

q3

array([[-1,  0,  0,  1],
       [ 0, -1,  0,  1],
       [ 0,  0, -1,  1],
       [ 0,  0,  0,  0]])

In [79]:
def matrix_multiplication(matrices):
    m = matrices[0]
    for i in range(1, len(matrices)):
        m = m @ matrices[i]
    
    return m

In [89]:
def generalised_likelihood(g_matrices, alphas, p_matrices, gamma):

    n_epochs = len(g_matrices)
    assert n_epochs == len(alphas)
    assert n_epochs == len(p_matrices)

    likelihoods = []
    
    for i in [0,1,2]:
        most_recent = g_matrices[0][i, 0:3]@alphas[0] + p_matrices[0][i, 0:3]
        middle = [g_matrices[n][i, 0:3]@alphas[n] + (1-matrix_multiplication(p_matrices[1:n])[i,3]) @ alphas[n] for n in range(1, n_epochs-1)]

        print(most_recent, middle, gamma)
        likelihoods[i] = most_recent @ middle * gamma 
    
    return np.sum(likelihoods)
    

In [90]:
def alpha_matrix(alpha, s_vals, t_start, t_end):
    return [alpha/(alpha+1) * (1/(alpha+1))**s 
            * np.exp(alpha * t_start)
     * (poisson.cdf(s, t_start*(alpha+1)) 
        - poisson.cdf(s, t_end*(alpha+1))) for s in s_vals] 

Trying first with working generator matrix implementation

In [91]:
q1, q2 = GeneratorMatrix.from_params([1,1,1,1,1,2,0.1,0.1,0.1,0.1,0.1])

In [92]:
eigenvectors = [q1.eigenvectors, q2.eigenvectors]
eigenvalues = [-q1.eigenvalues[0:3], -q2.eigenvalues[0:3]]

In [93]:
p_matrices = [linalg.expm(q.matrix) for q in [q1, q2]]

In [94]:
generalised_likelihood(eigenvectors, eigenvalues, p_matrices, gamma=1)

[1.287 0.955 1.011] [] 1


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 0 is different from 3)

In [99]:
g_matrices = eigenvectors
alphas = eigenvalues
p_matrices = [linalg.expm(q.matrix) for q in [q1, q2]]
n = 1
i=1

g_matrices[n][i, 0:3]@alphas[n] #+ (1-matrix_multiplication(p_matrices[0:n])[i, 3]) @ alphas[n] for n in range(1, n-1)]

-0.0070017369530878506

In [87]:
p_matrices

[array([[0.334, 0.001, 0.057, 0.607],
        [0.001, 0.334, 0.057, 0.607],
        [0.029, 0.029, 0.908, 0.035],
        [0.   , 0.   , 0.   , 1.   ]]),
 array([[0.334, 0.001, 0.057, 0.607],
        [0.001, 0.334, 0.057, 0.607],
        [0.029, 0.029, 0.908, 0.035],
        [0.   , 0.   , 0.   , 1.   ]])]